**420-A52-SF - Algorithmes d'apprentissage supervisé - Automne 2022 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2022 Mikaël Swawola
<br/>
![Travaux Pratiques - Validation croisée](static/11-A1-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif la mise en oeuvre de la validation croisée à k plis à l'aide de la librairie **scikit-learn**. Les modèles et algorithmes utilisés seront la régression logistique et la classification KNN Le jeu de données sera de nouveau **Heart**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## 0 - Chargement des bibliothèques

In [2]:
# Manipulation de données
import numpy as np
import pandas as pd

# Visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns

# Helpers
from helpers import polynomial

# Outils divers
from tqdm import tqdm
from collections import defaultdict

# Machine Learning
# Compléter au fur et à mesure du TP l'importation des modules scikit-learn requis

In [3]:
# Configuration de la visualisation
sns.set(style="darkgrid", rc={'figure.figsize':(12,6)})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

## 1 - Lecture du jeu de données

<strong style="color: #4a86e8">Exercice 1-1: lire le fichier `Heart.csv`<strong/>

In [4]:
# Compléter le code ci-dessous ~ 1-2 lignes
HRT = pd.read_csv("../../data/Heart.csv")

In [5]:
# On supprime tout de suite les données manquantes. Ceci sera vu plus en détail plus tard dans le cours
HRT = HRT.dropna()

<strong style="color: #4a86e8">Exercice 1-2: afficher les dix premières lignes de la trame de données HRT</strong>

In [6]:
# Compléter le code ci-dessous ~ 1 ligne
HRT.head()

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


## 2 - Préparation de données

In [7]:
HRT.isnull().sum()

Unnamed: 0    0
Age           0
Sex           0
ChestPain     0
RestBP        0
Chol          0
Fbs           0
RestECG       0
MaxHR         0
ExAng         0
Oldpeak       0
Slope         0
Ca            0
Thal          0
AHD           0
dtype: int64

Nous allons considérer l'ensemble des variables explicatives du jeu de données **Heart**

<strong style="color: #4a86e8">Exercice 2-1: Encoder les variables explicatives catégorielles. Utiliser le numpy array `X` pour stocker les variables explicatives et le vecteur `y` pour la variable réponse. Indice: utilisez pandas ;-)</strong>

In [8]:
# Compléter le code ci-dessous ~ 2-3 lignes
lstCol = ['ChestPain', 'Thal']
col_y = 'AHD'

HRT = HRT.loc[:, ~HRT.columns.str.contains('^Unnamed')]
X = HRT.drop(col_y, axis=1).copy()
y = HRT[col_y].copy()

HRT_numerique = pd.get_dummies(data=HRT, columns = lstCol, drop_first=True, prefix=lstCol).copy()
print(HRT.Thal.unique())
print(HRT.ChestPain.unique())

HRT_numerique = HRT_numerique.loc[:, ~HRT_numerique.columns.str.contains('^Unnamed')]
dfX = HRT_numerique.copy();
dfY = dfX[col_y]
dfX.drop(col_y, axis=1, inplace=True)

['fixed' 'normal' 'reversable']
['typical' 'asymptomatic' 'nonanginal' 'nontypical']


<strong style="color: #4a86e8">Exercice 2-1: Quel est le nombre de variables explicatives contenues dans `X` ?</strong>

In [35]:
# Compléter le code ci-dessous ~ 1 ligne  ---> 16
len(dfX.columns)

16

<strong style="color: #4a86e8">Exercice 2-2: Quel est le nombre d'observations ?</strong>

In [36]:
# Compléter le code ci-dessous ~ 1 ligne ---> 297
len(dfX)

297

## 3 - Validation croisée à 5 plis - Régression logistique

### 3 - 1 - Mélange des données et séparation en jeu d’entraînement et de test

<strong style="color: #4a86e8">Exercice 3-1: à l'aide de scikit-learn, sépararer les données en jeu d'entraînement et jeu de test. La taille du jeu de test doit représenter 30% de la taille du jeu de données et l'état du générateur aléatoire sera fixé à 2020 afin de permettre la reproductibilité</strong>

![3 - 1 - Mélange des données et séparation en jeu d’entraînement et de test](static/fig1.png)

In [12]:
# Compléter le code ci-dessous ~ 1 ligne
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfX,dfY, shuffle=True, test_size=0.30, random_state=2020)

Vérification des dimensions de `X_train` et `X_test`

In [13]:
print(X_train.shape)
print(X_test.shape)

(207, 16)
(90, 16)


### 3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur 5 plis de validation

<strong style="color: #4a86e8">Exercice 3-2: réaliser une validation croisée à 5-plis sur une régression logistique polynomiale (sans variables d'interaction) en faisant varier l'ordre `n` du polynôme de 1 à 5. Vous pouvez utiliser la fonction `polynomial` disponible dans `helpers.py`. Enregistrez dans le dictionnaire `history` les scores (accuracy) sur les jeux d'entraînement et de validation pour chaque valeurs de `n`</strong><br/><br/>

<strong style="color: green">Afin de faciliter l'écriture du code, cet exercice sera réalisé en deux étapes
<ul>
    <li>Etape 1: effectuer une validation croisée à 5 plis sur une régression logistique d'ordre 1 seulement</li>
    <li>Etape 2: inclure le code précédent dans une boucle for en faisant varier l'ordre de 1 à 5</li>
</ul>
<strong>

Instanciation de `scaler` pour la standardisation des données

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

### Etape 1 : effectuer une validation croisée à 5 plis sur une régression logistique d'ordre 1 seulement

![3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur k plis de validation](static/fig2.png)

[sklearn.model_selection.KFold(n_splits=5, shuffle=False, random_state=None)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

In [15]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

history = defaultdict(list)
# Pseudo code:
# ------------
#kf = KFold(n_splits = n_fold, shuffle=False)
n_fold=5
kf = KFold(n_splits = n_fold)

X_temp = polynomial(X_train, degree=1)[:,1:]
print(type(X_temp))
#Standardisation
scaler.fit(X_temp)
X_train_scaled = scaler.transform(X_temp)

for train_index, val_index in kf.split(X_train_scaled):
    print("train_index: ", train_index)
    #print("val_index: ", val_index)
    x_cv_train, X_cv_val = X_train_scaled[train_index,:], X_train_scaled[val_index,:]
    y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]
    #y_cv_train, y_cv_val = y_train[0], y_train[0]
    
    #clf = LogisticRegression(penalty="none", fit_intercept=True).fit(x_cv_train, y_cv_train)
    
    #history["train"].append(clf.score(x_cv_train, y_cv_train))
    #history["val"].append(clf.score(x_cv_index, y_cv_index))
    
#print(f"Accuracy de validation = {np.mean(history['val'])}")
#print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

<class 'numpy.ndarray'>
train_index:  [ 42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206]


KeyError: '[43, 50, 51, 52, 57, 58, 60, 61, 70, 72, 75, 77, 81, 83, 85, 87, 89, 93, 96, 101, 106, 107, 109, 110, 111, 114, 125, 129, 152, 153, 159, 164, 166, 172, 174, 175, 176, 182, 183, 184, 187, 191, 192, 193, 194, 200, 202, 204, 205] not in index'

In [28]:
### repetindo exemplo c/ reg linear em vez de logística
from sklearn.preprocessing import LabelEncoder

dfY_numerique = pd.DataFrame( LabelEncoder().fit_transform(dfY), columns = ['AHD'])

In [34]:
# Compléter le code ci-dessous ~ 10-20 lignes ...
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


X_train, X_test, y_train, y_test = train_test_split(dfX, dfY_numerique, test_size=0.3, random_state=2020)


history = defaultdict(list)

n_fold = 5
kf = KFold(n_splits = n_fold, shuffle=False)

#X1_train_poly = polynomial(Xtrain,1)

X_temp = dfX.copy()
scaler = StandardScaler()
sm.add_constant(X_temp)
scaler.fit(X_temp)
X_scaled = scaler.transform(X_temp)


for n in tqdm(range(1,n_fold)):
    # Compléter le code ci-dessous ~ quelques lignes ...
    X_temp = polynomial(X_train, degree=n)[:, 1:]
    
    scaler = StandardScaler()
    scaler.fit(X_temp)
    X_scaled = scaler.transform(X_temp)    
    
    clf_ordre = LinearRegression( fit_intercept=True).fit(X_scaled, y_train)
        
    
    Xn_test = polynomial(X_test, degree=n)[:, 1:]
    X_test_scaled = scaler.transform(Xn_test)
    
    
    score_train = clf_ordre.score(X_scaled, y_train)
    score_val = clf_ordre.score(X_test_scaled, y_test)
    
    
    history['train'].append(score_train)
    history['val'].append(score_val)
    
# Pseudo code:
# ------------
#
# Instancier KFold (sklearn)
#
# Ajouter x0 sur X
#
# Standardiser X
#
# Itérer sur les k plis:
#     Effectuer la classification sur le plis courant (jeu d'entraînement)
#     Évaluer les performances sur le plis de validation
    
print(f"Accuracy de validation = {np.mean(history['val'])}")
print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

100%|██████████| 4/4 [00:01<00:00,  3.06it/s]

Accuracy de validation = 0.25056431397946644
Accuracy d'entraînement = 0.5880713089425353


In [29]:
# Compléter le code ci-dessous ~ 10-20 lignes ...
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

history = defaultdict(list)

n_fold = 5
kf = KFold(n_splits = n_fold, shuffle=False)

#X1_train_poly = polynomial(Xtrain,1)

X_temp = dfX.copy()
scaler = StandardScaler()
sm.add_constant(X_temp)
scaler.fit(X_temp)
X_scaled = scaler.transform(X_temp)


for n in tqdm(range(1,n_fold)):
    # Compléter le code ci-dessous ~ quelques lignes ...
    X_temp = polynomial(X_train, degree=n)[:, 1:]
    
    scaler = StandardScaler()
    scaler.fit(X_temp)
    X_scaled = scaler.transform(X_temp)    
    
    clf_ordre = LogisticRegression(penalty="none", random_state=2020, fit_intercept=True, max_iter=30).fit(X_scaled, y_train)
        
    
    Xn_test = polynomial(X_test, degree=n)[:, 1:]
    X_test_scaled = scaler.transform(Xn_test)
    
    
    score_train = clf_ordre.score(X_scaled, y_train)
    score_val = clf_ordre.score(X_test_scaled, y_test)
    
    
    history['train'].append(score_train)
    history['val'].append(score_val)
    
# Pseudo code:
# ------------
#
# Instancier KFold (sklearn)
#
# Ajouter x0 sur X
#
# Standardiser X
#
# Itérer sur les k plis:
#     Effectuer la classification sur le plis courant (jeu d'entraînement)
#     Évaluer les performances sur le plis de validation
    
print(f"Accuracy de validation = {np.mean(history['val'])}")
print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

 25%|██▌       | 1/4 [00:01<00:03,  1.04s/it]/home/hadoop/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/hadoop/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

Accuracy de validation = 0.8388888888888889
Accuracy d'entraînement = 0.8864734299516909


### Etape 2 - inclure le code précédent dans une boucle for en faisant varier l'ordre de 1 à 5

![3 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur k plis de validation](static/fig3.png)

In [74]:
# Compléter le code ci-dessous ~ quelques lignes ...

def LogisticRegressionCV(X_train_scaled, y_train, n_splits=5):
    history = defaultdict(list)
    
    kf = KFold(n_splits)
    for train_index, val_index in kf.split(X_train_scaled):
        x_cv_train, x_cv_val = X_train_scaled[train_index,:], X_train_scaled[val_index,:]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]
    
        clf = LogisticRegression(solver="sag", max_iter=100, fit_intercept=True).fit(x_cv_train, y_cv_train)
    
        history["train"].append(clf.score(x_cv_train, y_cv_train))
        history["val"].append(clf.score(x_cv_val, y_cv_val))
    

history = defaultdict(list)
for n in range(1, 10):
    X_temp = polynomial(X_train, degree=n)[:, 1:]
    
    scaler.fit(X_temp)
    X_train_scaled = scaler.transform(X_temp)
    
    tr, val = LogisticRegressionCV(X_train_scaled, y_train)
    

KeyError: '[43, 50, 51, 52, 57, 58, 60, 61, 70, 72, 75, 77, 81, 83, 85, 87, 89, 93, 96, 101, 106, 107, 109, 110, 111, 114, 125, 129, 152, 153, 159, 164, 166, 172, 174, 175, 176, 182, 183, 184, 187, 191, 192, 193, 194, 200, 202, 204, 205] not in index'

### 3 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation

![3 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation](static/fig4.png)

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1,10), history['train'], label="train")
ax.plot(range(1,10), history['val'], label="test")
ax.set_xlabel('n', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 3 - 4 - Entrainement du meilleur modèle

<strong style="color: #4a86e8">Exercice 3-4 : utiliser les résultas précédents pour sélectionner le meilleur modèle, et réentrainer ce modèle sur l'ensemble des données d'entraînement</strong>

![3 - 4 - Entrainement du meilleur modèle](static/fig5.png)

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

None

### 3 - 5 - Performances du meilleur modèle

<strong style="color: #4a86e8">Exercice 3-5 : Évaluer les performances du meilleur modèle sur le jeu de test</strong>
![3 - 5 - Performances du modèle final](static/fig6.png)

In [75]:
# Compléter la fonction ci-dessous ~ quelques lignes ...
X_train, X_test, y_train, y_test = train_test_split(dfX,dfY, shuffle=True, test_size=0.30, random_state=2020)

## 4 - Validation croisée à 5 plis - Algorithme des k plus proches voisins

### 4 - 1 - Mélange des données et séparation en jeu d’entraînement et de test

Vous pouvez reprendre `X_train`, `X_test`, `y_train` et `y_test` obtenus au **3-1**

### 4 - 2 - Entraînement de plusieurs modèles de flexibilités différentes sur 5 plis de validation

<strong style="color: #4a86e8">Exercice 4-2: réaliser une validation croisée à k-plis sur une classification KNN en faisant varier le nombre de voisins `k` de 1 à 20. Enregistrez les scores (accuracy) sur les jeux d'entraînement et de validation pour chaque valeur de `k` dans le dictionnaire `history`</strong><br/><br/>
<strong style="color: green">Comme pour l'exercice 3, afin de faciliter l'écriture du code, cette question sera faite en deux étapes
<ul>
    <li>Etape 1: effectuer une validation croisée à 5 plis sur une classification KNN avec k=10 voisins seulement</li>
    <li>Etape 2: inclure le code précédent dans une boucle for en faisant varier le nombre de voisins de 1 à 20</li>
</ul>
<strong>

### Etape 1 - effectuer une validation croisée à 5 plis sur une classification KNN avec k=10 voisins seulement

In [82]:
# Compléter le code ci-dessous ~ 10-15 lignes ...

history = defaultdict(list) 
kf = KFold(5)
kf.get=n_splits(X_train, y_train)






#print(f"Accuracy de validation = {np.mean(history['val'])}")
#print(f"Accuracy d'entraînement = {np.mean(history['train'])}")

train: [ 42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113
 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206], test: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41]
train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  84  85 

### Etape 2 -  inclure le code précédent dans une boucle for en faisant varier le nombre de voisins de 1 à 20

In [ ]:
# Compléter le code ci-dessous ~ quelques lignes ...

def KNeighborsClassifierCV(k):
    history = defaultdict(list)
    None
    
history = defaultdict(list)
for k in range(1, 20):
    None

### 4 - 3 - Choix du meilleur modèle en fonction des performances sur les plis de validation

In [ ]:
f, ax = plt.subplots(1,1)
ax.plot(range(1, 20), history['train'], label="train")
ax.plot(range(1, 20), history['val'], label="test")
ax.set_xlabel('k', fontsize=14)
ax.set_ylabel('accuracy', fontsize=14)
ax.legend()

### 4 - 4 - Entraînement du meilleur modèle

<strong style="color: #4a86e8">Exercice 4-4: utiliser les résultas précédents pour sélectionner le meilleur modèle, et réentrainer ce modèle sur l'ensemble des données d'entraînement</strong>

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

None

### 4 - 5 - Performances du meilleur modèle

<strong style="color: #4a86e8">Exercice 4-5: Évaluer les performances du meilleur modèle sur le jeu de test</strong>

In [ ]:
# Compléter la fonction ci-dessous ~ quelques lignes ...

None

<strong style="color: #4a86e8">Exercice 4-6: Expliquer ces performances</strong>

### 5 - Choix du modèle final

<strong style="color: #4a86e8">Exercice 5: Quel modèle choisissez-vous ? Quels sont les valeurs des hyperparamètres ?</strong>

### Fin du TP